# Acabei de organizar meu VScode para trabalar com o conda, agora vou usar o DASK para criar SQLite
---
**Vamos usar dados de energia, porque sempre tive curiosidade para analisar dados de Geração de energia!**

In [15]:
#vamos inicialmente carregar nossas bibliotecas
import dask.dataframe as dd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy as sa
import sqlite3 as sql
import os
import re

In [16]:
#Agora que temos nosso dados vamos tentar salvar nossos dados em um SQL
df = dd.read_csv('global_power_plant_database.csv', dtype={'other_fuel3': 'object',
       'wepp_id': 'object'})

In [17]:
#Agora vamos ver como se sair o compute aqui
df.head()

,country,country_long,name,gppd_idnr,capacity_mw,latitude,longitude,primary_fuel,other_fuel1,other_fuel2,...,estimated_generation_gwh_2013,estimated_generation_gwh_2014,estimated_generation_gwh_2015,estimated_generation_gwh_2016,estimated_generation_gwh_2017,estimated_generation_note_2013,estimated_generation_note_2014,estimated_generation_note_2015,estimated_generation_note_2016,estimated_generation_note_2017
0,AFG,Afghanistan,Kajaki Hydroelectric Power Plant Afghanistan,GEODB0040538,33.0,32.322,65.1190,Hydro,NaN,NaN,...,123.77,162.90,97.39,137.76,119.50,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1
1,AFG,Afghanistan,Kandahar DOG,WKS0070144,10.0,31.670,65.7950,Solar,NaN,NaN,...,18.43,17.48,18.25,17.70,18.29,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE
2,AFG,Afghanistan,Kandahar JOL,WKS0071196,10.0,31.623,65.7920,Solar,NaN,NaN,...,18.64,17.58,19.10,17.62,18.72,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE,SOLAR-V1-NO-AGE
3,AFG,Afghanistan,Mahipar Hydroelectric Power Plant Afghanistan,GEODB0040541,66.0,34.556,69.4787,Hydro,NaN,NaN,...,225.06,203.55,146.90,230.18,174.91,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1
4,AFG,Afghanistan,Naghlu Dam Hydroelectric Power Plant Afghanistan,GEODB0040534,100.0,34.641,69.7170,Hydro,NaN,NaN,...,406.16,357.22,270.99,395.38,350.80,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1,HYDRO-V1


In [18]:
df.dtypes

country                            object
country_long                       object
name                               object
gppd_idnr                          object
capacity_mw                       float64
latitude                          float64
longitude                         float64
primary_fuel                       object
other_fuel1                        object
other_fuel2                        object
other_fuel3                        object
commissioning_year                float64
owner                              object
source                             object
url                                object
geolocation_source                 object
wepp_id                            object
year_of_capacity_data             float64
generation_gwh_2013               float64
generation_gwh_2014               float64
generation_gwh_2015               float64
generation_gwh_2016               float64
generation_gwh_2017               float64
generation_gwh_2018               

## A descrição das colunas do seguinte dataset pode ser visto em:
---
Fields:

	- `country` (text): 3 character country code corresponding to the ISO 3166-1 alpha-3 specification [5]
	- `country_long` (text): longer form of the country designation
	- `name` (text): name or title of the power plant, generally in Romanized form
	- `gppd_idnr` (text): 10 or 12 character identifier for the power plant
	- `capacity_mw` (number): electrical generating capacity in megawatts
	- `latitude` (number): geolocation in decimal degrees; WGS84 (EPSG:4326)
	- `longitude` (number): geolocation in decimal degrees; WGS84 (EPSG:4326)
	- `primary_fuel` (text): energy source used in primary electricity generation or export
	- `other_fuel1` (text): energy source used in electricity generation or export
	- `other_fuel2` (text): energy source used in electricity generation or export
	- `other_fuel3` (text): energy source used in electricity generation or export
	- `commissioning_year` (number): year of plant operation, weighted by unit-capacity when data is available
	- `owner` (text): majority shareholder of the power plant, generally in Romanized form
	- `source` (text): entity reporting the data; could be an organization, report, or document, generally in Romanized form
	- `url` (text): web document corresponding to the `source` field
	- `geolocation_source` (text): attribution for geolocation information
	- `wepp_id` (text): a reference to a unique plant identifier in the widely-used PLATTS-WEPP database.
	- `year_of_capacity_data` (number): year the capacity information was reported
	- `generation_gwh_2013` (number): electricity generation in gigawatt-hours reported for the year 2013
	- `generation_gwh_2014` (number): electricity generation in gigawatt-hours reported for the year 2014
	- `generation_gwh_2015` (number): electricity generation in gigawatt-hours reported for the year 2015
	- `generation_gwh_2016` (number): electricity generation in gigawatt-hours reported for the year 2016
	- `generation_gwh_2017` (number): electricity generation in gigawatt-hours reported for the year 2017
	- `generation_gwh_2018` (number): electricity generation in gigawatt-hours reported for the year 2018
	- `generation_gwh_2019` (number): electricity generation in gigawatt-hours reported for the year 2019
	- `generation_data_source` (text): attribution for the reported generation information
	- `estimated_generation_gwh_2013` (number): estimated electricity generation in gigawatt-hours for the year 2013 (see [2])
	- `estimated_generation_gwh_2014` (number): estimated electricity generation in gigawatt-hours for the year 2014 (see [2])
	- `estimated_generation_gwh_2015` (number): estimated electricity generation in gigawatt-hours for the year 2015 (see [2])
	- `estimated_generation_gwh_2016` (number): estimated electricity generation in gigawatt-hours for the year 2016 (see [2])
	- `estimated_generation_gwh_2017` (number): estimated electricity generation in gigawatt-hours for the year 2017 (see [2])
	- `estimated_generation_note_2013` (text): label of the model/method used to estimate generation for the year 2013 (see section on this field below)
	- `estimated_generation_note_2014` (text): label of the model/method used to estimate generation for the year 2014 (see section on this field below)
	- `estimated_generation_note_2015` (text): label of the model/method used to estimate generation for the year 2015 (see section on this field below)
	- `estimated_generation_note_2016` (text): label of the model/method used to estimate generation for the year 2016 (see section on this field below)
	- `estimated_generation_note_2017` (text): label of the model/method used to estimate generation for the year 2017 (see section on this field below)

In [19]:
#Vamos ver se conseguimos gerar um database desse arquivo
current_folder_name = "sqlite:///" + os.getcwd() + "\\global_power_plants.db"
df.to_sql('global_power_plant', current_folder_name, index = False, if_exists='replace')

In [20]:
#agora nossos dados estão salvos no SQLite, vamos tentar conectar
conector = sql.connect('global_power_plants.db')

#agora vamos puxar alguns dados só de brinks
query = '''SELECT country as Country, SUM(capacity_mw) as Capacidade
           FROM global_power_plant
           GROUP BY Country
           ORDER BY Capacidade DESC'''
df_agg = pd.read_sql_query(query, conector)
df_agg.head(10)

,Country,Capacidade
0,CHN,1.415067e+06
1,USA,1.204638e+06
2,IND,3.160885e+05
3,RUS,2.282201e+05
4,JPN,2.153659e+05
5,BRA,1.475893e+05
6,CAN,1.435787e+05
7,DEU,1.120404e+05
8,FRA,1.106159e+05
9,KOR,9.947268e+04


In [21]:
#fechando conexões
conector.close()

# Como sabemos que tudo funciona perfeitamente, vamos agora criar um Banco de Dados bem maior e mais parrudo!
---
## ***Banco que vamos criar vem dos dados de exportação e importação do Brasil disponível no link***
link: https://www.kaggle.com/daniellecd/dados-de-exportao-e-importao-do-brasil


![Exportação e Importação](https://www.fazcomex.com.br/blog/wp-content/uploads/2019/06/O-que-%C3%A9-importa%C3%A7%C3%A3o.png)

### Descrição do nosso dataset segue abaixo:
---
1. Descrição das tabelas EXP-COMPLETA e IMP-COMPLETA:
Arquivos CSV com separador ponto e vírgula (;) detalhado por ano, mês, código NCM, código da unidade estatística, código de país de destino/origem do produto, código da UF de origem/destino do produto, código da via de transporte, código da URF de embarque/desembarque, quantidade estatística, quilograma líquido, valor dólar FOB (US$).

2. Descrição das colunas e tabelas auxiliares:

* CO_ANO: Ano (1997 a 2021)

* CO_MES: Código do mês (1:Janeiro a 12:Dezembro)

* CO_NCM: Código da Nomenclatura Comum Mercosul - Utilizada para controle e identificação das mercadorias comercializadas no Brasil e nos outros países do Mercosul (cada NCM representa um produto diferente)

* CO_UNID: Código da Unidade de Medida Estatística que é uma unidade de medida padrão para cada NCM, podendo ter valores como quilograma, metro, litro, pares, tonelada e outros.

* CO_PAIS: Código do nome do país com a qual foi realizada a operação comercial (importação ou exportação)

* SG_UF _NCM (sigla UF origem/destino da NCM): Código da Unidade Federativa (estado) de origem (exportação) ou destino (importação) da mercadoria.

* CO_VIA: Código para identificação do meio de transporte utilizado (aéreo, marítimo, rodoviária, ferroviária e outros). Na exportação, é o método utilizado para o transporte de mercadorias entre o último local de embarque para o exterior. Na importação, configura-se através dos meios de acesso para os bens do primeiro ponto de entrada no território nacional.

* CO_URF (Unidade da Receita Federal): Código da agência responsável pela execução dos procedimentos necessários para o desembaraço aduaneiro da mercadoria importada/exportada

* QT_ESTAT: No detalhamento por NCM, cada produto tem sua unidade estatística. Grande parte dos produtos tem como unidade estatística o peso em quilogramas, mas existem outras: quilograma líquido, número (unidades), pares, dúzias, milheiro, tonelada. A tabela completa que relaciona cada NCM com sua unidade estatística pode ser encontrada em na tabela “NCM _ UNIDADE”. É importante ressaltar que não se deve somar quantidades estatísticas de NCMs que contenham unidades estatísticas diferentes. [2]

* KG_LIQUIDO: Medida que expressa o peso líquido da mercadoria. Mesmo produtos com quantidades estatísticas diferentes do quilograma também possuem disponível a medida em quilograma, referindo-se ao peso líquido da mercadoria, ou seja, mercadoria desconsiderando embalagens, caixas ou quaisquer outros adicionais de transporte. Vale relembrar que essa informação, bem como as demais informadas nas operações de comércio exterior, é de livre preenchimento e de responsabilidade exclusiva dos operadores de comércio exterior. [2]

* VL_FOB: O valor FOB indica o preço da mercadoria em dólares americanos (US$) sob o Incoterm FOB (Free on Board), modalidade na qual o vendedor é responsável por embarcar a mercadoria enquanto o comprador assume o pagamento do frete, seguros e demais custos pós embarque. [2]

In [29]:
#Verificando os nomes de arquivos disponíveis
local = os.getcwd() + '\\'+  os.listdir()[0]
print(local)

c:\Users\ademi\OneDrive\Documentos\Estudos Python\Projetos Aleatórios\dask\comex


In [30]:
#Pegando o nome dos arquivos
arquivos = os.listdir(local)
print(arquivos)

['CO_NCM.csv', 'CO_PAIS.csv', 'CO_UNID.csv', 'CO_URF.csv', 'CO_VIA.csv', 'database.db', 'EXP_COMPLETA.csv', 'IMP_COMPLETA.csv', 'NCM_UNIDADE.csv']


In [31]:
#Agora que pegamos o nome dos arquivos, vamos construir uma função que irá criar um banco de dados com os dados
def criando_banco(lista_arquivos, local):
    '''
    Essa função utiliza o nome da lista de arquivos e o local em que os arquivos estão alocados para realizar a criação de um
    database em sqlite para trabalhar com os dados!

    Lembre-se, essa função só funciona se tiver passado o local e a lista de arquivos e todos os arquivos estiverem em CSV
    '''
    database_local = local + '\\database.db' #criando o caminho do banco

    conector = sql.connect(database_local) #criando conexão com servidor SQLite

    for arquivo in lista_arquivos: #iterando arquivos
        if re.match('[A-Z_]*.csv', arquivo):
            local_arquivo = r'{}'.format(local + '\\' + arquivo).replace("\\", "/")
            print(local_arquivo)
            arquivo_ddf = dd.read_csv(local_arquivo, sep = ';', dtype = 'object') #carregando os arquivos
            nome = re.findall('([A-Za-z_]*).csv', arquivo)[0].lower()
            print(nome)
            arquivo_ddf.to_sql(nome, 'sqlite:///' + database_local, 
            index = False, if_exists='replace', chunksize = 1000000)



In [33]:
criando_banco(arquivos, local)


c:/Users/ademi/OneDrive/Documentos/Estudos Python/Projetos Aleatórios/dask/comex/CO_NCM.csv
co_ncm
c:/Users/ademi/OneDrive/Documentos/Estudos Python/Projetos Aleatórios/dask/comex/CO_PAIS.csv
co_pais
c:/Users/ademi/OneDrive/Documentos/Estudos Python/Projetos Aleatórios/dask/comex/CO_UNID.csv
co_unid
c:/Users/ademi/OneDrive/Documentos/Estudos Python/Projetos Aleatórios/dask/comex/CO_URF.csv
co_urf
c:/Users/ademi/OneDrive/Documentos/Estudos Python/Projetos Aleatórios/dask/comex/CO_VIA.csv
co_via
c:/Users/ademi/OneDrive/Documentos/Estudos Python/Projetos Aleatórios/dask/comex/EXP_COMPLETA.csv
exp_completa
c:/Users/ademi/OneDrive/Documentos/Estudos Python/Projetos Aleatórios/dask/comex/IMP_COMPLETA.csv
imp_completa
c:/Users/ademi/OneDrive/Documentos/Estudos Python/Projetos Aleatórios/dask/comex/NCM_UNIDADE.csv
ncm_unidade
